# LSTM Betting Strategy - Training Workflow

This notebook demonstrates the complete workflow for building an LSTM-based betting strategy using time-series odds data.

## Overview

We'll build a recurrent neural network that predicts win probability for NBA games using:
- **Time-series odds sequences** (72 hours of historical data sampled every 3 hours)
- **Line movement patterns** over time
- **Sharp vs retail odds discrepancies** evolution
- **Market dynamics** (bookmaker consensus, volatility)

The trained LSTM integrates seamlessly with the backtesting framework via `BetOpportunity.confidence`.

## Architecture

- **Input**: Historical odds sequences (batch_size, timesteps=24, features=14)
- **Model**: 2-layer LSTM → Fully Connected → Sigmoid activation
- **Output**: Win probability (0-1)

## Prerequisites

Ensure dependencies are installed:
```bash
uv add torch numpy matplotlib seaborn scikit-learn
```

In [ ]:
# Import required libraries
from datetime import datetime, timedelta
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import torch
import torch.nn as nn
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    roc_auc_score,
    roc_curve,
)
from sklearn.model_selection import train_test_split

# Project imports
from odds_analytics.backtesting import (
    BacktestConfig,
    BacktestEngine,
    BetConstraintsConfig,
    BetSizingConfig,
)
from odds_analytics.feature_extraction import SequenceFeatureExtractor
from odds_analytics.lstm_strategy import LSTMModel, LSTMStrategy
from odds_analytics.sequence_loader import (
    load_sequences_for_event,
    prepare_lstm_training_data,
)
from odds_core.config import Settings
from odds_core.database import async_session_maker
from odds_core.models import EventStatus
from odds_lambda.storage.readers import OddsReader

# Configure plotting
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (12, 6)

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Load settings
settings = Settings()

print("✓ Imports successful")
print(f"PyTorch version: {torch.__version__}")
print(f"Device: {torch.device('cuda' if torch.cuda.is_available() else 'cpu')}")

## 1. Data Preparation

Load historical events with final scores and prepare time-series sequences for LSTM training.

In [ ]:
# Configure training period
TRAIN_START = datetime(2025, 10, 23)
TRAIN_END = datetime(2025, 10, 31)

# LSTM hyperparameters
LOOKBACK_HOURS = 72  # Use 72 hours of historical data
TIMESTEPS = 24  # Sample every 3 hours (72/3 = 24 timesteps)
MARKET = "h2h"  # Focus on moneyline market

print(f"Loading events from {TRAIN_START.date()} to {TRAIN_END.date()}...")
print(f"Sequence configuration: {LOOKBACK_HOURS}h lookback, {TIMESTEPS} timesteps")

async with async_session_maker() as session:
    reader = OddsReader(session)
    
    # Get completed events
    events = await reader.get_events_by_date_range(
        start_date=TRAIN_START,
        end_date=TRAIN_END,
        status=EventStatus.FINAL
    )
    
    print(f"Found {len(events)} completed events")
    
    # Prepare LSTM training data
    # This creates sequences for both home and away teams
    X_home, y_home, masks_home = await prepare_lstm_training_data(
        events=events,
        session=session,
        market=MARKET,
        outcome="home",
        lookback_hours=LOOKBACK_HOURS,
        timesteps=TIMESTEPS,
    )
    
    X_away, y_away, masks_away = await prepare_lstm_training_data(
        events=events,
        session=session,
        market=MARKET,
        outcome="away",
        lookback_hours=LOOKBACK_HOURS,
        timesteps=TIMESTEPS,
    )

# Combine home and away samples
X = np.concatenate([X_home, X_away], axis=0)
y = np.concatenate([y_home, y_away], axis=0)
masks = np.concatenate([masks_home, masks_away], axis=0)

print(f"\nDataset shape: X={X.shape}, y={y.shape}, masks={masks.shape}")
print(f"Features per timestep: {X.shape[2]}")
print(f"Class distribution: {np.bincount(y)}")
print(f"Win rate: {y.mean():.2%}")

### Feature Overview

The `SequenceFeatureExtractor` creates **14 features per timestep**:

**Basic Odds Features**:
1. `american_odds` - Raw American odds
2. `decimal_odds` - Decimal odds
3. `implied_prob` - Market implied probability

**Line Movement**:
4. `odds_change_from_prev` - Change from previous timestep
5. `odds_change_from_opening` - Change from opening line
6. `prob_change_from_prev` - Probability change from previous
7. `prob_change_from_opening` - Probability change from opening

**Market Context**:
8. `num_bookmakers` - Number of bookmakers offering odds
9. `odds_std` - Standard deviation across bookmakers

**Sharp vs Retail**:
10. `sharp_odds` - Pinnacle/Circa odds (sharp bookmakers)
11. `sharp_prob` - Sharp implied probability
12. `retail_sharp_diff` - Retail vs sharp discrepancy

**Time Features**:
13. `hours_to_game` - Hours until game start
14. `time_of_day_sin` - Cyclical time encoding (sine)
15. `time_of_day_cos` - Cyclical time encoding (cosine)

In [ ]:
# Visualize a sample sequence
sample_idx = 0
sample_sequence = X[sample_idx]
sample_mask = masks[sample_idx]
sample_label = y[sample_idx]

# Get valid timesteps (where mask=1)
valid_steps = np.where(sample_mask == 1)[0]

# Plot key features over time
fig, axes = plt.subplots(3, 1, figsize=(14, 10))

# Plot 1: Implied probability evolution
axes[0].plot(valid_steps, sample_sequence[valid_steps, 2], marker='o', linewidth=2, label='Implied Prob')
axes[0].axhline(y=0.5, color='r', linestyle='--', alpha=0.5, label='50% Line')
axes[0].set_ylabel('Probability')
axes[0].set_title(f'Sample Sequence (Label: {"Win" if sample_label == 1 else "Loss"})')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot 2: Sharp vs Retail
axes[1].plot(valid_steps, sample_sequence[valid_steps, 10], marker='s', linewidth=2, label='Sharp Prob')
axes[1].plot(valid_steps, sample_sequence[valid_steps, 2], marker='o', linewidth=2, alpha=0.5, label='Market Prob')
axes[1].set_ylabel('Probability')
axes[1].set_title('Sharp vs Market Probability')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Plot 3: Hours to game
axes[2].plot(valid_steps, sample_sequence[valid_steps, 12], marker='x', linewidth=2, color='green')
axes[2].set_xlabel('Timestep')
axes[2].set_ylabel('Hours to Game')
axes[2].set_title('Time Until Game Start')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nSample sequence details:")
print(f"  Valid timesteps: {len(valid_steps)}/{TIMESTEPS}")
print(f"  Label: {'Win' if sample_label == 1 else 'Loss'}")
print(f"  Final implied prob: {sample_sequence[valid_steps[-1], 2]:.2%}")

## 2. Train/Test Split

Split data for training and evaluation. Note: For production, consider time-series split (walk-forward analysis).

In [ ]:
# Split data (80/20 train/test)
# Note: Using stratified random split for simplicity
# For production, consider time-based split to prevent lookahead bias
X_train, X_test, y_train, y_test, masks_train, masks_test = train_test_split(
    X, y, masks,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")
print(f"\nTraining win rate: {y_train.mean():.2%}")
print(f"Test win rate: {y_test.mean():.2%}")

# Convert to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train)
y_train_tensor = torch.FloatTensor(y_train)
masks_train_tensor = torch.FloatTensor(masks_train)

X_test_tensor = torch.FloatTensor(X_test)
y_test_tensor = torch.FloatTensor(y_test)
masks_test_tensor = torch.FloatTensor(masks_test)

print("\n✓ Data converted to PyTorch tensors")

## 3. Model Training

Train LSTM model using the LSTMStrategy class.

In [ ]:
# Initialize LSTM strategy
strategy = LSTMStrategy(
    lookback_hours=LOOKBACK_HOURS,
    timesteps=TIMESTEPS,
    hidden_size=64,
    num_layers=2,
    dropout=0.2,
    market=MARKET,
    min_edge_threshold=0.03,  # 3% minimum edge to bet
    min_confidence=0.52,  # 52% minimum model probability
)

print("LSTM Architecture:")
print(f"  Input size: {X.shape[2]} features")
print(f"  Hidden size: {strategy.hidden_size}")
print(f"  Number of layers: {strategy.num_layers}")
print(f"  Dropout: {strategy.dropout}")
print(f"  Total parameters: {sum(p.numel() for p in strategy.model.parameters()):,}")

# Training configuration
EPOCHS = 50
BATCH_SIZE = 16
LEARNING_RATE = 0.001

print(f"\nTraining configuration:")
print(f"  Epochs: {EPOCHS}")
print(f"  Batch size: {BATCH_SIZE}")
print(f"  Learning rate: {LEARNING_RATE}")

# Train the model
print("\nStarting training...")

# Prepare data for training
# Convert events back to BacktestEvent format for training
async with async_session_maker() as session:
    reader = OddsReader(session)
    events_for_training = await reader.get_events_by_date_range(
        start_date=TRAIN_START,
        end_date=TRAIN_END,
        status=EventStatus.FINAL
    )

history = await strategy.train(
    events=events_for_training,
    session=session,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    outcome="home"  # Train on home team outcomes
)

print("\n✓ Training complete")

### Training History

In [ ]:
# Plot training loss
plt.figure(figsize=(12, 6))
plt.plot(history['loss'], linewidth=2, label='Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss (BCE)')
plt.title('LSTM Training Loss Over Time')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Final training loss: {history['loss'][-1]:.4f}")
print(f"Initial loss: {history['loss'][0]:.4f}")
print(f"Loss reduction: {((history['loss'][0] - history['loss'][-1]) / history['loss'][0] * 100):.1f}%")

## 4. Model Evaluation

Evaluate model performance on held-out test set.

In [ ]:
# Make predictions on test set
strategy.model.eval()
with torch.no_grad():
    test_outputs = strategy.model(X_test_tensor, masks_test_tensor)
    y_pred_proba = test_outputs.numpy()
    y_pred = (y_pred_proba > 0.5).astype(int)

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)

print("Model Performance on Test Set:")
print(f"  Accuracy: {accuracy:.2%}")
print(f"  ROC-AUC: {roc_auc:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=["Loss", "Win"]))

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Loss', 'Win'], yticklabels=['Loss', 'Win'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix - LSTM Model')
plt.tight_layout()
plt.show()

### ROC Curve

In [ ]:
# Plot ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)

plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, linewidth=2, label=f'LSTM (AUC = {roc_auc:.4f})')
plt.plot([0, 1], [0, 1], 'k--', linewidth=1, label='Random Classifier')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve - LSTM Model')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### Calibration Plot

In [ ]:
# Plot calibration curve
from sklearn.calibration import calibration_curve

prob_true, prob_pred = calibration_curve(y_test, y_pred_proba, n_bins=10)

plt.figure(figsize=(10, 6))
plt.plot(prob_pred, prob_true, marker='o', linewidth=2, label='LSTM')
plt.plot([0, 1], [0, 1], 'k--', linewidth=1, label='Perfect Calibration')
plt.xlabel('Predicted Probability')
plt.ylabel('Actual Win Rate')
plt.title('Calibration Plot - LSTM Model')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\nNote: A well-calibrated model has predictions close to the diagonal line.")
print("If the model is consistently below the line, it's underconfident.")
print("If the model is consistently above the line, it's overconfident.")

### Prediction Distribution

In [ ]:
# Analyze prediction distribution
plt.figure(figsize=(12, 5))

# Subplot 1: Overall distribution
plt.subplot(1, 2, 1)
plt.hist(y_pred_proba, bins=20, edgecolor='black', alpha=0.7)
plt.axvline(x=0.5, color='r', linestyle='--', label='Decision Threshold')
plt.axvline(x=0.52, color='g', linestyle='--', label='Betting Threshold (min_confidence)')
plt.xlabel('Predicted Probability')
plt.ylabel('Frequency')
plt.title('Distribution of Predicted Probabilities')
plt.legend()
plt.grid(True, alpha=0.3)

# Subplot 2: By actual outcome
plt.subplot(1, 2, 2)
plt.hist(y_pred_proba[y_test == 0], bins=15, alpha=0.5, label='Actual Loss', edgecolor='black')
plt.hist(y_pred_proba[y_test == 1], bins=15, alpha=0.5, label='Actual Win', edgecolor='black')
plt.axvline(x=0.5, color='r', linestyle='--', alpha=0.5)
plt.xlabel('Predicted Probability')
plt.ylabel('Frequency')
plt.title('Predictions by Actual Outcome')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Statistics
print("Prediction Statistics:")
print(f"  Mean predicted probability: {y_pred_proba.mean():.4f}")
print(f"  Std of predictions: {y_pred_proba.std():.4f}")
print(f"  Min prediction: {y_pred_proba.min():.4f}")
print(f"  Max prediction: {y_pred_proba.max():.4f}")
print(f"\n  Predictions > 52% (would bet): {(y_pred_proba > 0.52).sum()} / {len(y_pred_proba)} ({(y_pred_proba > 0.52).mean():.1%})")

## 5. Save Model

Save the trained LSTM model for use in backtesting and production.

In [ ]:
# Create models directory if it doesn't exist
models_dir = Path("models")
models_dir.mkdir(exist_ok=True)

# Save model
model_path = models_dir / "lstm_h2h_v1.pth"
strategy.save_model(str(model_path))

print(f"✓ Model saved to {model_path}")
print("\nModel details:")
print(f"  Timesteps: {TIMESTEPS}")
print(f"  Features per timestep: {X.shape[2]}")
print(f"  Hidden size: {strategy.hidden_size}")
print(f"  Layers: {strategy.num_layers}")
print(f"  Training samples: {X_train.shape[0]}")
print(f"  Test ROC-AUC: {roc_auc:.4f}")
print(f"  Test Accuracy: {accuracy:.2%}")

## 6. Backtest the Strategy

Test the trained LSTM model on a separate time period using the backtesting framework.

In [ ]:
# Define backtest period (should be different from training period)
BACKTEST_START = TRAIN_END + timedelta(days=1)
BACKTEST_END = BACKTEST_START + timedelta(days=7)

# Load the saved model
backtest_strategy = LSTMStrategy(
    model_path=str(model_path),
    lookback_hours=LOOKBACK_HOURS,
    timesteps=TIMESTEPS,
    hidden_size=64,
    num_layers=2,
    dropout=0.2,
    market=MARKET,
    min_edge_threshold=0.03,  # Require 3% edge
    min_confidence=0.52,  # Only bet if model predicts >52% win probability
)

# Configure backtest with nested config objects
config = BacktestConfig(
    start_date=BACKTEST_START,
    end_date=BACKTEST_END,
    initial_bankroll=10000.0,
    decision_hours_before_game=1.0,
    sizing=BetSizingConfig(
        method="fractional_kelly",
        kelly_fraction=0.25,  # Quarter-Kelly for safety
    ),
    constraints=BetConstraintsConfig(
        min_bet_size=10.0,
        max_bet_size=500.0,
        max_bet_percentage=0.05
    ),
)

print(f"Running backtest from {BACKTEST_START.date()} to {BACKTEST_END.date()}...")

# Run backtest
async with async_session_maker() as session:
    reader = OddsReader(session)
    engine = BacktestEngine(backtest_strategy, config, reader)
    result = await engine.run()

print("\n✓ Backtest complete")
print(result.to_summary_text())

### Analyze Backtest Results

In [ ]:
# Save results
result.to_json("backtest_results_lstm.json")
result.to_csv("backtest_bets_lstm.csv")

print("✓ Results saved")
print("  JSON: backtest_results_lstm.json")
print("  CSV: backtest_bets_lstm.csv")

# Plot equity curve
import pandas as pd

equity_df = pd.DataFrame([
    {"date": point.date, "bankroll": point.bankroll}
    for point in result.equity_curve
])

plt.figure(figsize=(14, 6))
plt.plot(equity_df["date"], equity_df["bankroll"], linewidth=2, label='Bankroll')
plt.axhline(
    y=config.initial_bankroll,
    color='r',
    linestyle='--',
    alpha=0.5,
    label='Starting Bankroll'
)
plt.xlabel('Date')
plt.ylabel('Bankroll ($)')
plt.title('LSTM Strategy - Equity Curve')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\nKey Metrics:")
print(f"  Total Bets: {result.total_bets}")
print(f"  Win Rate: {result.win_rate:.2f}%")
print(f"  ROI: {result.roi:.2f}%")
print(f"  Total Profit: ${result.total_profit:,.2f}")
print(f"  Sharpe Ratio: {result.sharpe_ratio:.2f}")
print(f"  Max Drawdown: ${abs(result.max_drawdown):,.2f} ({result.max_drawdown_percentage:.2f}%)")

if result.profit_factor:
    print(f"  Profit Factor: {result.profit_factor:.2f}")
if result.calmar_ratio:
    print(f"  Calmar Ratio: {result.calmar_ratio:.2f}")

### Bet Analysis

In [ ]:
# Analyze bets by confidence level
if result.bet_records:
    bets_df = pd.DataFrame([vars(bet) for bet in result.bet_records])
    
    # Group by confidence bins
    bets_df['confidence_bin'] = pd.cut(bets_df['confidence'], bins=[0.5, 0.55, 0.6, 0.65, 1.0], 
                                        labels=['50-55%', '55-60%', '60-65%', '65%+'])
    
    confidence_analysis = bets_df.groupby('confidence_bin').agg({
        'event_id': 'count',
        'result': lambda x: (x == 'win').mean() * 100,
        'profit': 'sum',
        'stake': 'mean'
    }).round(2)
    
    confidence_analysis.columns = ['Count', 'Win Rate (%)', 'Total Profit ($)', 'Avg Stake ($)']
    
    print("\nPerformance by Confidence Level:")
    print(confidence_analysis)
    
    # Plot confidence vs actual performance
    plt.figure(figsize=(12, 5))
    
    plt.subplot(1, 2, 1)
    confidence_analysis['Win Rate (%)'].plot(kind='bar', color='steelblue', edgecolor='black')
    plt.axhline(y=52.38, color='r', linestyle='--', label='Breakeven (52.38%)')
    plt.ylabel('Win Rate (%)')
    plt.title('Win Rate by Confidence Level')
    plt.xticks(rotation=0)
    plt.legend()
    plt.grid(True, alpha=0.3, axis='y')
    
    plt.subplot(1, 2, 2)
    confidence_analysis['Total Profit ($)'].plot(kind='bar', color='green', edgecolor='black')
    plt.axhline(y=0, color='r', linestyle='-', linewidth=1)
    plt.ylabel('Total Profit ($)')
    plt.title('Profit by Confidence Level')
    plt.xticks(rotation=0)
    plt.grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.show()
else:
    print("\nNo bets placed during backtest period.")
    print("Consider:")
    print("  - Lowering min_confidence threshold")
    print("  - Lowering min_edge_threshold")
    print("  - Extending backtest period")

## 7. Summary

This notebook demonstrated:

1. **Data Preparation**: Loading historical events with time-series odds sequences
2. **Feature Engineering**: 14 features per timestep capturing line movement, sharp vs retail, and market dynamics
3. **Model Training**: 2-layer LSTM with dropout for binary win/loss prediction
4. **Evaluation**: ROC-AUC, calibration analysis, prediction distribution
5. **Backtesting**: Seamless integration with backtesting framework using Kelly Criterion sizing

### Key Insights

**LSTM Advantages**:
- Captures **temporal patterns** in line movement
- Learns **sequential dependencies** (e.g., steam moves, reverse line movement)
- Handles **variable-length sequences** via attention masks
- Naturally incorporates **time-until-game** dynamics

**Compared to XGBoost**:
- **XGBoost**: Better for tabular features, interpretable, faster training
- **LSTM**: Better for sequential patterns, handles time-series naturally, requires more data

### Next Steps

**Model Improvements**:
1. **Attention Mechanism**: Add attention layers to focus on critical timesteps (e.g., closing line)
2. **Bidirectional LSTM**: Process sequences forward and backward
3. **Multi-Task Learning**: Predict both winner and margin of victory
4. **Ensemble**: Combine LSTM with XGBoost for robust predictions

**Feature Engineering**:
1. **Team Features**: Recent form, injuries, rest days, travel distance
2. **Matchup Features**: Head-to-head history, pace metrics, defensive ratings
3. **Advanced Line Movement**: Volume-weighted odds, steam detection, reverse line movement flags

**Training Enhancements**:
1. **Walk-Forward Validation**: Time-series cross-validation
2. **Hyperparameter Tuning**: Grid search for hidden_size, num_layers, dropout
3. **Learning Rate Scheduling**: Decay learning rate over epochs
4. **Early Stopping**: Monitor validation loss to prevent overfitting

**Production Deployment**:
1. **Real-Time Inference**: Integrate with live odds fetching
2. **Model Versioning**: Track model performance over time
3. **A/B Testing**: Compare multiple model versions in production
4. **Monitoring**: Track prediction calibration and bet performance

### Resources

- **LSTM Implementation**: `packages/odds-analytics/odds_analytics/lstm_strategy.py`
- **Feature Extraction**: `packages/odds-analytics/odds_analytics/feature_extraction.py`
- **Sequence Loading**: `packages/odds-analytics/odds_analytics/sequence_loader.py`
- **Backtesting Guide**: `BACKTESTING_GUIDE.md`